# Proyecto de Data Science - Parte III
# **AUTOPISTAS AUSA**

# Definición del objetivo del proyecto

El objetivo principal de este proyecto es analizar los siniestros viales ocurridos en las autopistas de la Ciudad de Buenos Aires,
con el fin de identificar patrones significativos que permitan no solo comprender las características y la frecuencia de los accidentes,
sino también predecir las características de futuros siniestros viales. Para lograr esto, se empleará un modelo de clasificación basado en
Random Forest, un algoritmo de aprendizaje automático que es particularmente útil para la predicción y la clasificación en grandes volúmenes de datos.


# Contexto comercial

En la Ciudad de Buenos Aires, las autopistas representan una parte crucial de la infraestructura vial, siendo arterias principales para
el flujo vehicular. Sin embargo, las autopistas también presentan desafíos significativos relacionados con la seguridad vial. La
frecuencia de los accidentes viales en las autopistas, especialmente aquellos que resultan en víctimas fatales o lesiones graves, tiene
repercusiones significativas para los residentes de la ciudad, los conductores y los servicios de emergencia.

Este proyecto tiene como fin proporcionar insights clave sobre los factores que contribuyen a los accidentes y su frecuencia. De esta manera,
se busca equipar a las autoridades locales con la información necesaria para tomar decisiones informadas y reducir los riesgos y costos operativos.


# Problema comercial

El principal problema comercial que enfrenta la Ciudad de Buenos Aires y sus autoridades viales es la falta de un sistema eficiente para
predecir el comportamiento de los siniestros viales en las autopistas de la ciudad. Sin esta capacidad predictiva, se dificulta la
implementación de medidas preventivas y correctivas basadas en datos concretos.

Este proyecto busca llenar ese vacío, proporcionando un modelo predictivo basado en datos históricos que ayudará a predecir las características
de los siniestros, como la ubicación, la gravedad, la hora y las condiciones de tráfico, lo que permitirá a las autoridades implementar
acciones preventivas más efectivas.


# Contexto analítico

Este proyecto se apoya en un dataset detallado que contiene información sobre los siniestros viales ocurridos en las autopistas de la Ciudad
de Buenos Aires. El dataset incluye una amplia gama de variables que son clave para el análisis y la predicción de accidentes, tales como:

- **Condiciones climáticas:** Información sobre si el accidente ocurrió bajo lluvia, niebla, altas temperaturas u otras condiciones meteorológicas adversas.
- **Tipo de vehículo involucrado:** Detalles sobre los vehículos que participaron en los accidentes (por ejemplo, autos particulares, camiones, motocicletas).
- **Horario del accidente:** Información detallada sobre la hora exacta en la que ocurrió cada siniestro.
- **Número de víctimas:** Información sobre las víctimas, tanto fatales como lesionadas, que estuvieron involucradas en cada accidente.
- **Autopistas específicas:** Información sobre las autopistas donde ocurrieron los siniestros, lo cual es crucial para identificar puntos críticos o zonas con mayor frecuencia de accidentes.

El análisis de este dataset permitirá descubrir patrones relevantes que puedan predecir las características de los accidentes y su gravedad.


In [1]:
# Importación de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [5]:
# Carga del Dataset
from google.colab import files

uploaded = files.upload()
dataset_path = list(uploaded.keys())[0]

Saving dsdatasetagustin.xlsx to dsdatasetagustin (1).xlsx


In [8]:
import pandas as pd
data = pd.read_excel(dataset_path)

In [9]:
# Exploración inicial del dataset
print("Primeras filas del dataset:")
print(data.head())

print("\nInformación general del dataset:")
print(data.info())

print("\nEstadísticas descriptivas del dataset:")
print(data.describe())

Primeras filas del dataset:
       fecha  hora         autopista  clima superficie  lesionados  \
0 2023-01-02     8  DIST. 9 DE JULIO  BUENO       SECA           1   
1 2023-01-03    14       AV. CANTILO  BUENO       SECA           1   
2 2023-01-03    14     AU DELLEPIANE  BUENO       SECA           0   
3 2023-01-05    14       AU FRONDIZI  BUENO       SECA           1   
4 2023-01-05    17     AU 25 DE MAYO  BUENO       SECA           1   

   fallecidos                                     siniestro  moto  liviano  \
0           0  SINIESTRO DE UN SOLO VEHICULO / SIN COLISION     1        0   
1           0              COLISION CON DOS O MAS VEHICULOS     0        2   
2           0                             OBSTACULO NO FIJO     0        1   
3           0              COLISION CON DOS O MAS VEHICULOS     1        1   
4           0              COLISION CON DOS O MAS VEHICULOS     1        1   

   bus  camion  
0    0       0  
1    0       0  
2    0       0  
3    0       0

In [11]:
# Identificación de valores nulos
print("\nValores nulos por columna:")
print(data.isnull().sum())


Valores nulos por columna:
fecha         0
hora          0
autopista     0
clima         0
superficie    0
lesionados    0
fallecidos    0
siniestro     0
moto          0
liviano       0
bus           0
camion        0
dtype: int64


In [12]:
# Preparación de datos
X = data.drop(columns=['fallecidos', 'lesionados', 'fecha', 'siniestro'])
y = data['lesionados']

# Conversión de variables categóricas
X = pd.get_dummies(X, drop_first=True)

# Selección de características
k_best = SelectKBest(score_func=chi2, k=5)
X_new = k_best.fit_transform(X, y)
selected_features = k_best.get_support(indices=True)
print("Características seleccionadas:")
print(X.columns[selected_features])

Características seleccionadas:
Index(['hora', 'moto', 'liviano', 'autopista_AU PERITO MORENO',
       'clima_LLUVIOSO'],
      dtype='object')


In [13]:
# División en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42)

# Entrenamiento del modelo
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predicciones
y_pred = clf.predict(X_test)

In [14]:
# Evaluación del modelo
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))

print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

print("\nPrecisión del modelo: {:.2f}%".format(accuracy_score(y_test, y_pred) * 100))

Matriz de confusión:
[[228  30   2   0   0   0]
 [ 35  69   3   0   0   0]
 [  5   8   1   0   0   0]
 [  2   0   0   0   0   0]
 [  2   0   0   0   1   0]
 [  0   0   0   0   0   1]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.84      0.88      0.86       260
           1       0.64      0.64      0.64       107
           2       0.17      0.07      0.10        14
           3       0.00      0.00      0.00         2
           4       1.00      0.33      0.50         3
           5       1.00      1.00      1.00         1

    accuracy                           0.78       387
   macro avg       0.61      0.49      0.52       387
weighted avg       0.76      0.78      0.76       387


Precisión del modelo: 77.52%


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [15]:
# Conclusiones
print("### Conclusiones")
print("""
1. Las características seleccionadas por el método SelectKBest incluyen: {}.
   Estas características resultaron ser las más relevantes para predecir el número de lesionados, y pueden ser clave para comprender los factores que influyen en los accidentes.
2. El modelo Random Forest alcanzó una precisión del {:.2f}% en el conjunto de prueba, lo que demuestra su capacidad para clasificar correctamente los datos y predecir incidentes con base en las variables seleccionadas.
3. Los resultados sugieren que variables como las condiciones climáticas y el horario del accidente tienen una relación significativa con el número de víctimas.
""".format(X.columns[selected_features], accuracy_score(y_test, y_pred) * 100))

# Recomendaciones
print("### Recomendaciones")
print("""
1. Implementar medidas preventivas en horarios identificados como de alto riesgo, por ejemplo, en franjas nocturnas o durante condiciones climáticas adversas.
2. Monitorear con mayor frecuencia las autopistas específicas que presentan un mayor número de siniestros, y evaluar la implementación de medidas como mayor señalización o controles de velocidad.
3. Utilizar los insights obtenidos para optimizar los recursos de emergencia, priorizando áreas y horarios críticos para reducir los tiempos de respuesta en accidentes.
4. Ampliar el análisis con más datos, como factores socioeconómicos, datos del tráfico y reportes de mantenimiento vial, para enriquecer el modelo y generar predicciones más robustas.
5. Comunicar los hallazgos a las autoridades pertinentes para diseñar campañas de concientización sobre seguridad vial dirigidas a conductores.
""")


### Conclusiones

1. Las características seleccionadas por el método SelectKBest incluyen: Index(['hora', 'moto', 'liviano', 'autopista_AU PERITO MORENO',
       'clima_LLUVIOSO'],
      dtype='object').
   Estas características resultaron ser las más relevantes para predecir el número de lesionados, y pueden ser clave para comprender los factores que influyen en los accidentes.
2. El modelo Random Forest alcanzó una precisión del 77.52% en el conjunto de prueba, lo que demuestra su capacidad para clasificar correctamente los datos y predecir incidentes con base en las variables seleccionadas.
3. Los resultados sugieren que variables como las condiciones climáticas y el horario del accidente tienen una relación significativa con el número de víctimas.

### Recomendaciones

1. Implementar medidas preventivas en horarios identificados como de alto riesgo, por ejemplo, en franjas nocturnas o durante condiciones climáticas adversas.
2. Monitorear con mayor frecuencia las autopistas específ